---

## 🎓 Summary: Functional Programming in Python+Polars

### Key Takeaways:

1. **Result<T,E>** → Explicit error handling without exceptions  
2. **Option<T>** → Null safety without defensive checks  
3. **Thunk<T>** → Lazy evaluation with memoization  
4. **Polars Lazy** → Rust-powered deferred execution  
5. **pl.when()** → Railway-oriented programming  

### Performance Benefits:

- **Polars lazy queries**: 10-100x faster than Pandas (Rust optimization)  
- **Streaming**: Handle datasets larger than RAM  
- **Zero-copy**: No data duplication in memory  
- **SIMD**: Vectorized operations via Rust  

### When to Use:

- ✅ **Large datasets** (>1GB) → Polars lazy + streaming  
- ✅ **Complex pipelines** → Result/Option for error handling  
- ✅ **Type safety** → Explicit nullability and errors  
- ✅ **Performance-critical** → Rust-compiled transformations  

### Further Reading:

- [Polars User Guide](https://docs.pola.rs)  
- [Railway-Oriented Programming](https://fsharpforfunandprofit.com/rop/)  
- [Rust Result Type](https://doc.rust-lang.org/std/result/)

---

**💡 Remember**: Functional programming isn't about purity for purity's sake.  
It's about **composability**, **predictability**, and **performance**.

**Polars gives you Rust-level speed with Python-level ergonomics. 🚀**

In [ ]:
# Simulate trading signals
signals = pl.DataFrame({
    "symbol": ["AAPL", "GOOGL", "MSFT", "TSLA", "AAPL", "GOOGL"],
    "signal": ["buy", "sell", "buy", "buy", "sell", "buy"],
    "price": [150.0, 2800.0, 300.0, 700.0, 155.0, 2750.0],
    "confidence": [0.95, 0.60, 0.85, 0.45, 0.90, 0.70]
})

print("🔥 Combined Functional Pipeline:")
print()

# Wrap DataFrame creation in Result for safety
def load_signals() -> Result:
    """Safe data loader"""
    try:
        return Result.ok(signals)
    except Exception as e:
        return Result.err(f"Failed to load: {e}")

# Safe processing pipeline
def process_signals(df: pl.DataFrame) -> Result:
    """Process with error handling"""
    try:
        result = (df.lazy()
                  # Filter high-confidence signals
                  .filter(pl.col("confidence") > 0.65)
                  # Categorize by signal type
                  .with_columns([
                      pl.when(pl.col("signal") == "buy")
                        .then(pl.col("price") * 1.1)  # Target price
                        .otherwise(pl.col("price") * 0.9)
                        .alias("target_price")
                  ])
                  # Group by symbol
                  .group_by("symbol")
                  .agg([
                      pl.count().alias("num_signals"),
                      pl.col("confidence").mean().alias("avg_confidence"),
                      pl.col("target_price").mean().alias("avg_target")
                  ])
                  .sort("avg_confidence", descending=True)
                  .collect())
        
        return Result.ok(result)
    except Exception as e:
        return Result.err(f"Processing failed: {e}")

# Execute pipeline with monadic composition
final_result = (load_signals()
                .and_then(process_signals)
                .map(lambda df: df.to_dict()))

# Pattern match on result
final_result.match(
    on_ok=lambda data: print(f"✅ Success!\n{pl.from_dict(data)}"),
    on_err=lambda err: print(f"❌ Error: {err}")
)

---

## 🔥 Part 6: Combining Monads + Polars for Real-World Use Case

**Scenario**: Process trading signals with error handling  
**Challenge**: File I/O, parsing, filtering, aggregation - all can fail  
**Solution**: Result monad for I/O + Polars lazy for data processing

### Example: Safe Signal Processing Pipeline

In [ ]:
# Railway-oriented data transformations
transactions = pl.DataFrame({
    "amount": [100, -50, 200, -30, 0, 500, -100],
    "currency": ["USD", "EUR", "USD", "GBP", "USD", "EUR", "USD"],
    "verified": [True, True, False, True, True, False, True]
})

print("🌊 Railway-Oriented Data Pipeline:")
print()

result = transactions.with_columns([
    # Safe amount categorization
    pl.when(pl.col("amount") == 0)
      .then(pl.lit("zero"))
      .when(pl.col("amount") > 0)
      .then(pl.lit("credit"))
      .otherwise(pl.lit("debit"))
      .alias("type"),
    
    # Risk assessment with multiple conditions
    pl.when(~pl.col("verified"))
      .then(pl.lit("high_risk"))
      .when((pl.col("amount").abs() > 300) & pl.col("verified"))
      .then(pl.lit("medium_risk"))
      .otherwise(pl.lit("low_risk"))
      .alias("risk_level"),
    
    # Currency conversion (safe division embedded)
    pl.when(pl.col("currency") == "USD")
      .then(pl.col("amount"))
      .when(pl.col("currency") == "EUR")
      .then(pl.col("amount") * 1.1)
      .when(pl.col("currency") == "GBP")
      .then(pl.col("amount") * 1.3)
      .otherwise(pl.col("amount"))
      .alias("amount_usd")
])

print(result)

---

## 🌊 Part 5: Railway-Oriented Programming with Polars

**Pattern**: `pl.when().then().otherwise()` = Rust's match expressions  
**Benefit**: Explicit branching with no hidden nulls  
**Performance**: Compiled to Rust - faster than Python if/else

### Example: Complex Business Logic

In [ ]:
# Create sample data
df = pl.DataFrame({
    "user_id": range(1, 101),
    "score": np.random.randint(0, 100, 100),
    "timestamp": [datetime.now()] * 100
})

print("⚡ Polars Lazy Evaluation:")
print()

# Lazy query - builds plan but doesn't execute
lazy_query = (df.lazy()
              .filter(pl.col("score") > 50)
              .with_columns([
                  (pl.col("score") * 2).alias("doubled_score"),
                  pl.when(pl.col("score") > 75)
                    .then(pl.lit("high"))
                    .when(pl.col("score") > 50)
                    .then(pl.lit("medium"))
                    .otherwise(pl.lit("low"))
                    .alias("category")
              ])
              .group_by("category")
              .agg([
                  pl.count().alias("count"),
                  pl.col("doubled_score").mean().alias("avg_doubled")
              ]))

print(f"LazyFrame: {type(lazy_query)}")
print("Query plan:")
print(lazy_query.explain())
print()

# Execute with .collect() - like Thunk.force()
result = lazy_query.collect()
print("📊 Executed result:")
print(result)

---

## ⚡ Part 4: Polars Lazy Evaluation - Built-in Functional Programming

**Key Insight**: Polars' `.lazy()` IS a Rust-powered Thunk!  
**Pattern**: Deferred execution with query optimization  
**Benefit**: Faster than eager execution, uses less memory

### Example: Lazy DataFrame Pipeline

In [ ]:
# Thunk for lazy evaluation
import time

def expensive_computation():
    """Simulates expensive operation"""
    print("  🔄 Computing... (this only prints once!)")
    time.sleep(0.5)
    return sum(range(1000000))

print("⏱️ Lazy Evaluation Demo:")
print()

# Create thunk - doesn't execute yet
lazy_value = Thunk(expensive_computation)
print(f"Thunk created: {lazy_value}")
print(f"Is evaluated? {lazy_value.is_evaluated()}")
print()

# First force - executes
print("First call to .force():")
result1 = lazy_value.force()
print(f"  → Result: {result1}")
print(f"  → Evaluated: {lazy_value.is_evaluated()}")
print()

# Second force - returns cached
print("Second call to .force():")
result2 = lazy_value.force()
print(f"  → Result: {result2} (instant!)")
print()

# Map is also lazy
lazy_doubled = lazy_value.map(lambda x: x * 2)
print(f"Mapped thunk: {lazy_doubled}")
print(f"  → {lazy_doubled.force()}")

---

## ⏱️ Part 3: Thunk<T> - Lazy Evaluation with Memoization

**Concept**: Defer computation until needed, cache result  
**Pattern**: Lazy evaluation for expensive operations  
**Benefit**: Performance optimization + referential transparency

### Example: Expensive Computation

In [ ]:
# Option for safe dictionary access
config = {
    "host": "localhost",
    "port": 8080,
    "timeout": 30
}

print("🔍 Safe Data Access with Option:")
print()

# Success case
host = safe_access(config, "host")
print(f"Host: {host} → {host.unwrap()}")

# Missing key
database = safe_access(config, "database")
print(f"Database: {database} → {database.unwrap_or('default.db')}")
print()

# Chaining with map and filter
port_info = (safe_access(config, "port")
             .map(lambda p: p * 2)
             .filter(lambda p: p > 8000)
             .map(lambda p: f"High port: {p}"))

print(f"🎯 Port transformation: {port_info}")
print(f"  → {port_info.unwrap_or('No high port found')}")
print()

# Pattern matching
timeout_message = safe_access(config, "timeout").match(
    on_some=lambda t: f"Timeout set to {t}s",
    on_nothing=lambda: "No timeout configured"
)
print(f"⏱️ {timeout_message}")

---

## 🎯 Part 2: Option<T> Monad - Null Safety

**Concept**: Explicit optionality without None checks everywhere  
**Pattern**: Map/flatMap/filter compose elegantly  
**Benefit**: No NullPointerException - errors caught at composition time

### Example: Safe Data Access

In [ ]:
# Safe division and sqrt pipeline
print("🌊 Railway-Oriented Programming Demo:")
print()

# Success case: 100 / 25 = 4.0, sqrt(4.0) = 2.0, * 2 = 4.0
success = safe_compute_pipeline(100, 25)
print(f"Pipeline(100, 25): {success}")
print(f"  → Result: {success.unwrap()}")
print()

# Error case 1: Division by zero
error1 = safe_compute_pipeline(100, 0)
print(f"Pipeline(100, 0): {error1}")
print(f"  → Handled: {error1.unwrap_or(-1)}")
print()

# Error case 2: Negative sqrt
error2 = safe_compute_pipeline(-100, 5)
print(f"Pipeline(-100, 5): {error2}")
print(f"  → Handled: {error2.unwrap_or(-1)}")
print()

# Chaining with .and_then() - errors automatically propagate!
result = (safe_divide(100, 25)
          .and_then(safe_sqrt)
          .map(lambda x: x * 2)
          .map(lambda x: f"Final: {x}"))
          
print(f"🎯 Chained result: {result.unwrap()}")

---

## 🦀 Part 1: Result<T,E> Monad - Rust-Style Error Handling

**Concept**: Explicit error handling without exceptions  
**Pattern**: Railway-oriented programming - errors "skip" success track  
**Benefit**: Errors are values you can map/chain/compose

### Example: Safe Mathematical Pipeline

In [ ]:
# Setup imports
import sys
sys.path.append('/Users/melvinalvarez/Documents/Workspace/polaroid/notebooks')

from functional_monads import Result, Option, Thunk
from functional_monads import safe_divide, safe_sqrt, safe_access, safe_compute_pipeline

import polars as pl
import numpy as np
from datetime import datetime

print(f"✅ Functional Monads imported | Polars {pl.__version__}")
print(f"🦀 Result, Option, Thunk ready for functional programming!")

# 🧬 Polaroid Functional Programming Showcase

**Rust Monads + Polars Streaming + Railway-Oriented Programming**

---

This notebook demonstrates **production-grade functional programming** combining:

🦀 **Rust-Inspired Monads** - Result<T,E>, Option<T>, Thunk<T>  
⚡ **Polars Streaming** - Lazy evaluation with query optimization  
🌊 **Railway-Oriented Programming** - Safe error propagation  
🎯 **Pure Functions** - Immutable data transformations  
🔒 **Type Safety** - Explicit nullability and error handling  

**Target Audience**: Scala/Haskell/Rust developers discovering Python+Polars  
**Goal**: Show that Python can do functional programming at Rust speeds

---